# Test point-based registration

In [21]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
from scipy.spatial.transform import Rotation as R
import open3d as o3d
import copy

import planeslam.geometry as geometry
import planeslam.io as io
from planeslam.scanrep import ScanRep, pc_to_scan

Read in airsim LiDAR and pose data

In [2]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [3]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Open3D ICP registration

In [22]:
# Visualization helper
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [23]:
# Initialize source and target
source = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(PC_data[0])

target = o3d.geometry.PointCloud()
target.points = o3d.utility.Vector3dVector(PC_data[1])

In [24]:
# Visualize
o3d.visualization.draw_geometries([source, target])

In [25]:
# Initial guess transform 
trans_init = np.eye(4)
threshold = 0.02

evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

RegistrationResult with fitness=4.595588e-04, inlier_rmse=1.658737e-02, and correspondence_set size of 3
Access transformation to get result.


In [26]:
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=6.280637e-03, inlier_rmse=1.329315e-02, and correspondence_set size of 41
Access transformation to get result.
Transformation is:
[[ 9.62607006e-01  7.05381633e-05  2.70901728e-01 -9.08054546e-01]
 [-1.56330597e-04  9.99999944e-01  2.95113544e-04 -6.75400308e-03]
 [-2.70901692e-01 -3.26428594e-04  9.62606964e-01 -4.43856240e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
